# Introduction to Data Science 


## Lab 8: Scrapping and Integration

**British University in Egypt**<br>
**Instructors:** Nahla Barakat
---

In [1]:
from IPython.display import HTML
style = "<style>div.exercise { background-color: #ffcccc;border-color: #E9967A; border-left: 5px solid #800080; padding: 0.5em;}</style>"
HTML(style)

In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import math

C:\Users\Mostafa\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Data Scraping and Preparation 

**In this lab, we will scrap data related to solar flares phenomena from two different web sources, then we will integrate the data to retrieve extra columns (Features)**

## First page: Scraping Solar flares from space Weather page

In [3]:
# In Sime pages, Some tages does not appears directly, so we need  to define some headers 
# and using request to request the page
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
url='https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares'
spaceWeather = requests.get(url,headers=headers)

In [4]:
# Getting the text from the page and use BS4 to parse it
spaceWeather_text = spaceWeather.text
spaceWeather_page = BeautifulSoup(spaceWeather_text,'html.parser')

In [5]:
# Showing first 1000 element in the text
spaceWeather_page.prettify()[:1000]

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <title>\n   Top 50 solar flares | Solar activity | SpaceWeatherLive.com\n  </title>\n  <meta charset="utf-8"/>\n  <meta content="index, follow, max-snippet:-1, max-image-preview:large, max-video-preview:-1" name="robots"/>\n  <meta content="On this page you will find an overview of the strongest solar flares since June 1996 together with links to more information in our archive and a v..." name="description"/>\n  <meta content="SpaceWeatherLive, Live, Aurora, Auroral activity, Aurora Australis, Aurora Borealis, northern lights, Solar wind, Kp-index, Space Weather, Space Weather Updates, Aurora forecast, Space Weather Alerts, Solar activity, Solar flares, Sunspots, Aurora alert, Auroral activity, The Sun, SDO, STEREO, EPAM, DSCOVR" name="keywords"/>\n  <!-- Facebook meta -->\n  <meta content="https://spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html" property="og:url"/>\n  <meta content="article" property="og:type"/>\n  <meta 

**You should determine the part that includes the data to retrieve this part.<br>
After checking our page, we need to retrieve a table that contains the solar flares information**

In [6]:
len(spaceWeather_page.find('table'))

2

**Getting Table tags not enough, since there are two tables in the page, we need a marker to get the needed table** 

In [7]:
# We found that the required table with this class name
class_='table table-striped'
table = spaceWeather_page.find('table',attrs={'class':class_})
table

<table class="table table-striped"><thead><tr><th></th><th></th><th></th><th>Region</th><th>Start</th><th>Maximum</th><th>End</th><th></th></tr></thead><tbody><tr><td>1</td><td><span class="flare_small Xclass">X28+</span></td><td>2003/11/04</td><td><a class="btn btn-SWL RegionBGD" href="/en/solar-activity/region/10486.html">0486</a></td><td>19:29</td><td>19:53</td><td>20:06</td><td><div class="btn-group"><a class="btn btn-xs btn-outline-dark" data-fancybox="" href="/images/Archief/2003/Flares/200311041953X28.0.mp4"><i aria-hidden="true" class="far fa-play-circle"></i> Movie</a><a class="btn btn-xs btn-outline-dark" href="/en/archive/2003/11/04/xray.html">View archive</a></div></td></tr><tr><td>2</td><td><span class="flare_small Xclass">X20+</span></td><td>2001/04/02</td><td><a class="btn btn-SWL RegionBGD" href="/en/solar-activity/region/9393.html">9393</a></td><td>21:32</td><td>21:51</td><td>22:03</td><td><div class="btn-group"><a class="btn btn-xs btn-outline-dark" data-fancybox="" h

In [8]:
# Checking the type of the table
type(table)

bs4.element.Tag

In [9]:
# Printing the table headers to convert the table to pandas dataframe
table.find_all('th')

[<th></th>,
 <th></th>,
 <th></th>,
 <th>Region</th>,
 <th>Start</th>,
 <th>Maximum</th>,
 <th>End</th>,
 <th></th>]

In [10]:
# seperate the data into lists
rank = []
x_class = []
date = []
region = []
start_time = []
max_time = []
end_time = []
movie = []

rows = table.find_all('tr')
rows[:3]

[<tr><th></th><th></th><th></th><th>Region</th><th>Start</th><th>Maximum</th><th>End</th><th></th></tr>,
 <tr><td>1</td><td><span class="flare_small Xclass">X28+</span></td><td>2003/11/04</td><td><a class="btn btn-SWL RegionBGD" href="/en/solar-activity/region/10486.html">0486</a></td><td>19:29</td><td>19:53</td><td>20:06</td><td><div class="btn-group"><a class="btn btn-xs btn-outline-dark" data-fancybox="" href="/images/Archief/2003/Flares/200311041953X28.0.mp4"><i aria-hidden="true" class="far fa-play-circle"></i> Movie</a><a class="btn btn-xs btn-outline-dark" href="/en/archive/2003/11/04/xray.html">View archive</a></div></td></tr>,
 <tr><td>2</td><td><span class="flare_small Xclass">X20+</span></td><td>2001/04/02</td><td><a class="btn btn-SWL RegionBGD" href="/en/solar-activity/region/9393.html">9393</a></td><td>21:32</td><td>21:51</td><td>22:03</td><td><div class="btn-group"><a class="btn btn-xs btn-outline-dark" data-fancybox="" href="/images/Archief/2001/Flares/200104022151X20.0

In [11]:
i = 0 
for row in rows:
    if i == 0: # skip the header
        i += 1
        continue
    cols = row.find_all('td')
    print(cols)
    break

[<td>1</td>, <td><span class="flare_small Xclass">X28+</span></td>, <td>2003/11/04</td>, <td><a class="btn btn-SWL RegionBGD" href="/en/solar-activity/region/10486.html">0486</a></td>, <td>19:29</td>, <td>19:53</td>, <td>20:06</td>, <td><div class="btn-group"><a class="btn btn-xs btn-outline-dark" data-fancybox="" href="/images/Archief/2003/Flares/200311041953X28.0.mp4"><i aria-hidden="true" class="far fa-play-circle"></i> Movie</a><a class="btn btn-xs btn-outline-dark" href="/en/archive/2003/11/04/xray.html">View archive</a></div></td>]


In [12]:
i = 0 
for row in rows:
    if i == 0: # skip the header
        i += 1
        continue
    cols = row.find_all('td')
    rank.append(cols[0].text)
    x_class.append(cols[1].text)
    date.append(cols[2].text)
    region.append(cols[3].text)
    start_time.append(cols[4].text)
    max_time.append(cols[5].text)
    end_time.append(cols[6].text)
    movie.append(cols[7].text)
len(rank), len(x_class), len(date), len(region), len(start_time), len(max_time), len(end_time), len(movie)

(50, 50, 50, 50, 50, 50, 50, 50)

In [13]:
# create a dataframe
spaceWeatherDataFrame = pd.DataFrame({'Rank':rank,'X_class':x_class,'Date':date,'Region':region,'Start_time':start_time,'Max_time':max_time,'End_time':end_time,'Movie':movie})
spaceWeatherDataFrame.head()

,Rank,X_class,Date,Region,Start_time,Max_time,End_time,Movie
0,1,X28+,2003/11/04,0486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,0486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,0808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


In [14]:
len(spaceWeatherDataFrame)

50

## Step 2: Tidy the First dataframe

In [15]:
# drop the Movie column
modifiedDataFrame=spaceWeatherDataFrame
modifiedDataFrame.drop(['Movie'],axis=1,inplace=True)
modifiedDataFrame.head()

,Rank,X_class,Date,Region,Start_time,Max_time,End_time
0,1,X28+,2003/11/04,0486,19:29,19:53,20:06
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03
2,3,X17.2+,2003/10/28,0486,09:51,11:10,11:24
3,4,X17+,2005/09/07,0808,17:17,17:40,18:03
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55


### Adding the date to Start_time, End_time, Max_time columns 
- Compining the string values of the **Date** column and each value of the **time** columns
- Dropping the **Date** column

In [16]:
modifiedDataFrame['Start_time'] = modifiedDataFrame['Date'] + " " + modifiedDataFrame['Start_time']
modifiedDataFrame['End_time'] = modifiedDataFrame['Date'] + " " + modifiedDataFrame['End_time']
modifiedDataFrame['Max_time'] = modifiedDataFrame['Date'] + " " + modifiedDataFrame['Max_time']
modifiedDataFrame.drop(['Date'], axis=1, inplace=True)
modifiedDataFrame.head()

,Rank,X_class,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003/11/04 19:29,2003/11/04 19:53,2003/11/04 20:06
1,2,X20+,9393,2001/04/02 21:32,2001/04/02 21:51,2001/04/02 22:03
2,3,X17.2+,0486,2003/10/28 09:51,2003/10/28 11:10,2003/10/28 11:24
3,4,X17+,0808,2005/09/07 17:17,2005/09/07 17:40,2005/09/07 18:03
4,5,X14.4,9415,2001/04/15 13:19,2001/04/15 13:50,2001/04/15 13:55


### Converting the Time columns from String to Pandas TimeStamp
- using `pd.to_datetime`

In [17]:
modifiedDataFrame['Start_time'] = pd.to_datetime(modifiedDataFrame['Start_time'])
modifiedDataFrame['End_time'] = pd.to_datetime(modifiedDataFrame['End_time'])
modifiedDataFrame['Max_time'] = pd.to_datetime(modifiedDataFrame['Max_time'])
modifiedDataFrame[:10]

,Rank,X_class,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20+,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2+,0486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17+,0808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00
4,5,X14.4,9415,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00
5,6,X10,0486,2003-10-29 20:37:00,2003-10-29 20:49:00,2003-10-29 21:01:00
6,7,X9.4,8100,1997-11-06 11:49:00,1997-11-06 11:55:00,1997-11-06 12:01:00
7,8,X9.3,2673,2017-09-06 11:53:00,2017-09-06 12:02:00,2017-09-06 12:10:00
8,9,X9,0930,2006-12-05 10:18:00,2006-12-05 10:35:00,2006-12-05 10:45:00
9,10,X8.3,0486,2003-11-02 17:03:00,2003-11-02 17:25:00,2003-11-02 17:39:00


In [18]:
modifiedDataFrame.dtypes

Rank                  object
X_class               object
Region                object
Start_time    datetime64[ns]
Max_time      datetime64[ns]
End_time      datetime64[ns]
dtype: object

In [19]:
# check for missing values
modifiedDataFrame.isnull().sum()

Rank          0
X_class       0
Region        0
Start_time    0
Max_time      0
End_time      0
dtype: int64

## Step3: Scrape the NASA data

In [20]:
# get the website content
reb = requests.get("http://www.hcbravo.org/IntroDataSci/misc/waves_type2.html")
reb.text[:1000]

'<html><body>\n<h2>Wind/WAVES type II bursts and CMEs</h2>\n<a href="waves_type2_description.htm">A Brief Description</a>\n<pre>\nNOTE: List includes DH type II bursts observed by Wind spacecraft, \nbut after STEREO launch on Oct 2006 the start and end times and \nfrequencies of bursts are determined using both Wind and STEREO \nobservations\n\n=================================================================================================\n                DH Type II                       Flare                     CME                   \n----------------------------------------   -----------------   --------------------------   Plots\nStart            End          Frequency     Loc   NOAA  Imp    Date  Time CPA  Width  Spd        \n(1)        (2)   (3)   (4)   (5)    (6)     (7)    (8)  (9)    (10)  (11) (12)  (13) (14)   (15) \n=================================================================================================\n1997/04/01 14:00 04/01 14:15  <a href="http://cdaw.gsfc.nas

In [21]:
page=BeautifulSoup(reb.text)
page.prettify()[:1000]

'<html>\n <body>\n  <h2>\n   Wind/WAVES type II bursts and CMEs\n  </h2>\n  <a href="waves_type2_description.htm">\n   A Brief Description\n  </a>\n  <pre>\nNOTE: List includes DH type II bursts observed by Wind spacecraft, \nbut after STEREO launch on Oct 2006 the start and end times and \nfrequencies of bursts are determined using both Wind and STEREO \nobservations\n\n=================================================================================================\n                DH Type II                       Flare                     CME                   \n----------------------------------------   -----------------   --------------------------   Plots\nStart            End          Frequency     Loc   NOAA  Imp    Date  Time CPA  Width  Spd        \n(1)        (2)   (3)   (4)   (5)    (6)     (7)    (8)  (9)    (10)  (11) (12)  (13) (14)   (15) \n=================================================================================================\n1997/04/01 14:00 04/01 14:15  <a

### Using Regex to scrap the content of the NASA website
- getting the rows of the website divided into lists of values
- e.g. `['1997/04/01', '14:00', '04/01', '14:15', '8000', '4000', 'S25E16', '8026', 'M1.3', '04/01', '15:18', '74', '79', '312', 'PHTX']`

In [22]:
rex=r'\d+/.*'
l=re.findall(rex,page.text)
l

['1997/04/01 14:00 04/01 14:15  8000  4000   S25E16  8026 M1.3   04/01 15:18   74   79  312   PHTX',
 '1997/04/07 14:30 04/07 17:30 11000  1000   S28E19  8027 C6.8   04/07 14:27 Halo  360  878   PHTX',
 '1997/05/12 05:15 05/14 16:00 12000    80   N21W08  8038 C1.3   05/12 05:30 Halo  360  464   PHTX',
 '1997/05/21 20:20 05/21 22:00  5000   500   N05W12  8040 M1.3   05/21 21:00  263  165  296   PHTX',
 '1997/09/23 21:53 09/23 22:16  6000  2000   S29E25  8088 C1.4   09/23 22:02  133  155  712   PHTX',
 '1997/11/03 05:15 11/03 12:00 14000   250   S20W13  8100 C8.6   11/03 05:28  240  109  227   PHTX',
 '1997/11/03 10:30 11/03 11:30 14000  5000   S16W21  8100 M4.2   11/03 11:11  233  122  352   PHTX',
 '1997/11/04 06:00 11/05 04:30 14000   100   S14W33  8100 X2.1   11/04 06:10 Halo  360  785   PHTX',
 '1997/11/06 12:20 11/07 08:30 14000   100   S18W63  8100 X9.4   11/06 12:10 Halo  360 1556   PHTX',
 '1997/11/27 13:30 11/27 14:00 14000  7000   N17E63  8113 X2.6   11/27 13:56   98   91  441

In [23]:
newList=[]
for i in l:
    newList.append(i.split())
newList[0]

['1997/04/01',
 '14:00',
 '04/01',
 '14:15',
 '8000',
 '4000',
 'S25E16',
 '8026',
 'M1.3',
 '04/01',
 '15:18',
 '74',
 '79',
 '312',
 'PHTX']

In [24]:
# put the values into seperated lists
startDate=[]
startTime=[]
endDate=[]
endTime=[]
startFrequency=[]
endFrequency=[]
flare_Location=[]
flare_region=[]
importance=[]
CME_Date=[]
CME_Time=[]
CPA=[]
width = []
speed = []
for i in newList:
    startDate.append(i[0])
    startTime.append(i[1])
    endDate.append(i[2])
    endTime.append(i[3])
    startFrequency.append(i[4])
    endFrequency.append(i[5])
    flare_Location.append(i[6])
    flare_region.append(i[7])
    importance.append(i[8])
    CME_Date.append(i[9])
    CME_Time.append(i[10])
    CPA.append(i[11])
    width.append(i[12])
    speed.append(i[13])

In [25]:
# put the lists into Pandas df
df = pd.DataFrame({'startDate':startDate,'startTime':startTime,'endDate':endDate,
                   'endTime':endTime,'startFrequency':startFrequency,'endFrequency':endFrequency,
                   'flare_Location': flare_Location,'flare_region':flare_region,'importance':importance,
                   'CME_Date':CME_Date,'CME_Time':CME_Time, 'width':width, 'speed':speed, 'CPA':CPA})
df.head()

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
0,1997/04/01,14:00,04/01,14:15,8000,4000,S25E16,8026,M1.3,04/01,15:18,79,312,74
1,1997/04/07,14:30,04/07,17:30,11000,1000,S28E19,8027,C6.8,04/07,14:27,360,878,Halo
2,1997/05/12,05:15,05/14,16:00,12000,80,N21W08,8038,C1.3,05/12,05:30,360,464,Halo
3,1997/05/21,20:20,05/21,22:00,5000,500,N05W12,8040,M1.3,05/21,21:00,165,296,263
4,1997/09/23,21:53,09/23,22:16,6000,2000,S29E25,8088,C1.4,09/23,22:02,155,712,133


In [26]:
len(df.columns)

14

In [27]:
df.columns

Index(['startDate', 'startTime', 'endDate', 'endTime', 'startFrequency',
       'endFrequency', 'flare_Location', 'flare_region', 'importance',
       'CME_Date', 'CME_Time', 'width', 'speed', 'CPA'],
      dtype='object')

### Missing Values per Column
- missing values: has `-` in it 

In [28]:
df.isna().sum()

startDate         0
startTime         0
endDate           0
endTime           0
startFrequency    0
endFrequency      0
flare_Location    0
flare_region      0
importance        0
CME_Date          0
CME_Time          0
width             0
speed             0
CPA               0
dtype: int64

In [29]:
missing_cols = []
for col in df.columns:
    s = df[col].str.contains('-').sum()
    print(col,':', s)
    if s>0:
        missing_cols.append(col)

startDate : 0
startTime : 0
endDate : 0
endTime : 0
startFrequency : 0
endFrequency : 0
flare_Location : 6
flare_region : 83
importance : 105
CME_Date : 20
CME_Time : 20
width : 20
speed : 20
CPA : 21


In [30]:
# columns with missing values
missing_cols

['flare_Location',
 'flare_region',
 'importance',
 'CME_Date',
 'CME_Time',
 'width',
 'speed',
 'CPA']

In [31]:
# discovering rows with missing data
df[df[col].str.contains('-')]

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
96,2000/11/09,16:15,11/11,04:00,10000,40,S11E10,9221,M1.0,--/--,--:--,----,----,----
149,2001/10/09,11:20,10/09,11:55,14000,2000,S28E08,9653,----,--/--,--:--,----,----,----
150,2001/10/09,13:10,10/09,23:00,5000,50,BACK,-----,----,--/--,--:--,----,----,----
229,2003/07/10,14:10,07/10,16:10,4000,400,N13W90,10397,M3.6,--/--,--:--,----,----,----
250,2004/01/05,03:40,01/05,03:50,9000,2500,S12E38,10536,M6.9,--/--,--:--,----,----,----
261,2004/06/22,22:07,06/22,22:30,10000,7000,S12W24,10635,C1.7,--/--,--:--,----,----,----
262,2004/06/23,06:30,06/23,08:55,14000,5000,S09W21,10635,C2.5,--/--,--:--,----,----,----
269,2004/09/19,17:15,09/19,18:15,14000,2500,N03W58,10672,M1.9,--/--,--:--,----,----,----
284,2005/01/04,11:20,01/04,11:35,13000,6000,N05W11,10715,C7.3,--/--,--:--,----,----,----
298,2005/06/16,20:25,06/16,21:40,9000,1000,N08W90,10775,M4.0,--/--,--:--,----,----,----


## Step 4: Tidy the NASA dataframe

### Replace any missing entries as `nan` in all columns

In [32]:
new_df=df.copy()

In [33]:
# replace nan
for col in df.columns:
    new_df.loc[new_df[col].str.contains('-'), col] = math.nan

In [34]:
new_df.isna().sum()

startDate           0
startTime           0
endDate             0
endTime             0
startFrequency      0
endFrequency        0
flare_Location      6
flare_region       83
importance        105
CME_Date           20
CME_Time           20
width              20
speed              20
CPA                21
dtype: int64

In [35]:
new_df.dropna(inplace=True)

In [36]:
new_df[10:20]

,startDate,startTime,endDate,endTime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
10,1997/12/12,22:45,12/12,23:20,14000,8000,N25W52,8116,B9.4,12/13,00:26,73,191,278
11,1998/01/25,15:03,01/25,15:18,14000,10000,N21E25,8141,C1.1,01/25,15:26,360,693,Halo
13,1998/04/20,10:25,04/22,06:00,10000,35,S22W90,8194,M1.4,04/20,10:07,165,1863,284
14,1998/04/23,06:00,04/23,15:30,14000,200,S17E90,8210,X1.2,04/23,05:55,360,1691,Halo
15,1998/04/24,09:17,04/24,09:25,4700,2600,S10E90,8210,C8.9,04/24,08:55,84,1184,100
16,1998/04/27,09:20,04/27,10:00,10000,1000,S16E50,8210,X1.0,04/27,08:56,360,1385,Halo
17,1998/04/29,16:30,04/29,17:00,10000,2000,S18E20,8210,M6.8,04/29,16:58,360,1374,Halo
18,1998/05/02,14:25,05/02,14:50,5000,3000,S15W15,8210,X1.1,05/02,14:06,360,938,Halo
19,1998/05/06,08:25,05/06,08:35,14000,5000,S11W65,8210,X2.7,05/06,08:29,190,1099,309
20,1998/05/09,03:35,05/09,10:00,9000,400,S14W89,8210,M7.7,05/09,03:35,178,2331,262


### Compine Date and Time columns for Start, End and CME
- endDate: getting the years value from **startDate** column before compining it with **endTime** column
- startDate: compining **startDate** with **startTime** columns

In [37]:
new_df['endDate'] = new_df['startDate'].str.slice(start=0, stop=5) + new_df['endDate'] + ' ' + new_df['endTime']
new_df['startDate'] = new_df['startDate'] + ' ' + new_df['startTime']

In [38]:
# drop endTime and startTime columns
new_df.drop(['endTime', 'startTime'], axis=1, inplace=True)

# rename
new_df.rename(columns={'startDate': 'Start_Datetime', 'endDate':'End_Datetime'}, inplace=True)

In [39]:
new_df.dtypes

Start_Datetime    object
End_Datetime      object
startFrequency    object
endFrequency      object
flare_Location    object
flare_region      object
importance        object
CME_Date          object
CME_Time          object
width             object
speed             object
CPA               object
dtype: object

## Custom Function for datetime conversion
- some time data had `24` hours instead of `0` in it so we made a special case for it 
 - By setting the `hours` to `0` and increamenting days by `1`

In [40]:
def custom_to_datetime(date):
    # If the time is 24, set it to 0 and increment day by 1
    if date[11:13] == '24':
        return pd.to_datetime(date[:-5]) + pd.Timedelta(days=1)
    else:
        return pd.to_datetime(date)

In [41]:
# applying custom_to_datetime function to time columns
for col in ['Start_Datetime', 'End_Datetime']:
    new_df[col] = new_df[col].apply(custom_to_datetime)

In [42]:
new_df.dtypes

Start_Datetime    datetime64[ns]
End_Datetime      datetime64[ns]
startFrequency            object
endFrequency              object
flare_Location            object
flare_region              object
importance                object
CME_Date                  object
CME_Time                  object
width                     object
speed                     object
CPA                       object
dtype: object

## Data Integration

After **Data Collection**, **Data Exploration**, and **Data Preprocessing** Now, the next step is to combine data from different sources to get a unified structure with more meaningful and valuable information. <br> This is mostly used if the data is segregated into different sources. This process of bringing data together in one place is called **data integration**.

#### The most used solution to integrate the data is combining dataframes using the merge function. <br>
[pd.merge documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

### Important Arguments
**how** {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’

Type of merge to be performed.

    left: use only keys from left frame.

    right: use only keys from right frame.

    outer: use union of keys from both frames.

    inner: use intersection of keys from both frames.

**on**: label or list

Column or index level names to join on. These must be found in both DataFrames. If on is None and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.

**left_on**: label or list, or array-like

Column or index level names to join on in the left DataFrame. Can also be an array or list of arrays of the length of the left DataFrame. These arrays are treated as if they are columns.

**right_on** :label or list, or array-like

Column or index level names to join on in the right DataFrame. Can also be an array or list of arrays of the length of the right DataFrame. These arrays are treated as if they are columns.

**left_index**: bool, default False
Use the index from the left DataFrame as the join key(s). If it is a MultiIndex, the number of keys in the other DataFrame (either the index or a number of columns) must match the number of levels

**right_index**: bool, default False
Use the index from the right DataFrame as the join key. Same caveats as left_index.


#### Before mergeing, you should makesure that the column you will merge on has the same format in the two frames

In [43]:
# Explore first dataframe
frame1=new_df.copy()
frame1.tail(20)

,Start_Datetime,End_Datetime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
450,2014-04-02 13:42:00,2014-04-03 08:10:00,14000,60,N11E53,12027,M6.5,04/02,13:36,360,1471,Halo
452,2014-04-04 14:02:00,2014-04-04 14:07:00,14000,11000,N13E26,12027,C8.3,04/04,14:12,96,467,54
453,2014-04-18 13:05:00,2014-04-18 22:50:00,14000,150,S20W34,12036,M7.3,04/18,13:25,360,1203,Halo
458,2014-06-10 12:58:00,2014-06-10 15:00:00,16000,1000,S17E82,12087,X1.5,06/10,13:30,360,1469,Halo
459,2014-06-12 22:14:00,2014-06-12 22:35:00,14000,6000,S20W55,12085,M3.1,06/12,22:12,186,684,233
460,2014-07-30 07:44:00,2014-07-30 08:00:00,6300,4500,N10E30,EP?,C1.5,07/30,07:00,254,700,13
461,2014-08-01 18:58:00,2014-08-02 05:00:00,1000,150,S10E11,12127,M1.5,08/01,18:36,360h,789,Halo
462,2014-08-22 10:37:00,2014-08-22 11:18:00,14000,6000,N12E01,12146,C2.2,08/22,11:12,360,600,Halo
463,2014-08-25 15:20:00,2014-08-25 16:02:00,14000,4000,N05W36,12146,M2.0,08/25,15:36,360,555,Halo
464,2014-08-25 20:43:00,2014-08-25 21:00:00,14000,7200,N07W43,12146,M3.9,08/25,20:48,177,711,273


In [44]:
# Explore Second dataframe
frame2=spaceWeatherDataFrame.copy()
frame2.head()

,Rank,X_class,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20+,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2+,0486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17+,0808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00
4,5,X14.4,9415,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00


**We need to rename column that we will merge on**

In [45]:
frame2.rename(columns={'X_class': 'importance'}, inplace=True)

In [46]:
frame2.head()

,Rank,importance,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20+,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2+,0486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17+,0808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00
4,5,X14.4,9415,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00


#### we have a problem here

In [47]:
frame2[frame2['importance'].str.contains('X.*\+')]

,Rank,importance,Region,Start_time,Max_time,End_time
0,1,X28+,0486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20+,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2+,0486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17+,0808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00


In [48]:
frame1[frame1['importance'].str.contains('X')].head()

,Start_Datetime,End_Datetime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA
7,1997-11-04 06:00:00,1997-11-05 04:30:00,14000,100,S14W33,8100,X2.1,11/04,06:10,360,785,Halo
8,1997-11-06 12:20:00,1997-11-07 08:30:00,14000,100,S18W63,8100,X9.4,11/06,12:10,360,1556,Halo
9,1997-11-27 13:30:00,1997-11-27 14:00:00,14000,7000,N17E63,8113,X2.6,11/27,13:56,91,441,98
14,1998-04-23 06:00:00,1998-04-23 15:30:00,14000,200,S17E90,8210,X1.2,04/23,05:55,360,1691,Halo
16,1998-04-27 09:20:00,1998-04-27 10:00:00,10000,1000,S16E50,8210,X1.0,04/27,08:56,360,1385,Halo


In [49]:
x=frame2['importance'][frame2['importance'].str.contains('X.*\+')]
x

0      X28+
1      X20+
2    X17.2+
3      X17+
Name: importance, dtype: object

In [50]:
x.str.slice(start=0, stop=-1)

0      X28
1      X20
2    X17.2
3      X17
Name: importance, dtype: object

In [51]:
frame2['importance'][frame2['importance'].str.contains('X.*\+')]=x.str.slice(start=0, stop=-1)

<ipython-input-51-577b3b02fc19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame2['importance'][frame2['importance'].str.contains('X.*\+')]=x.str.slice(start=0, stop=-1)


In [52]:
frame2.head()

,Rank,importance,Region,Start_time,Max_time,End_time
0,1,X28,0486,2003-11-04 19:29:00,2003-11-04 19:53:00,2003-11-04 20:06:00
1,2,X20,9393,2001-04-02 21:32:00,2001-04-02 21:51:00,2001-04-02 22:03:00
2,3,X17.2,0486,2003-10-28 09:51:00,2003-10-28 11:10:00,2003-10-28 11:24:00
3,4,X17,0808,2005-09-07 17:17:00,2005-09-07 17:40:00,2005-09-07 18:03:00
4,5,X14.4,9415,2001-04-15 13:19:00,2001-04-15 13:50:00,2001-04-15 13:55:00


In [53]:
## Now we are ready to integrate
pd.merge(frame1, frame2, on = 'importance',how='inner')

,Start_Datetime,End_Datetime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA,Rank,Region,Start_time,Max_time,End_time
0,1997-11-06 12:20:00,1997-11-07 08:30:00,14000,100,S18W63,8100,X9.4,11/06,12:10,360,1556,Halo,7,8100,1997-11-06 11:49:00,1997-11-06 11:55:00,1997-11-06 12:01:00
1,1997-11-27 13:30:00,1997-11-27 14:00:00,14000,7000,N17E63,8113,X2.6,11/27,13:56,91,441,98,49,0720,2005-01-15 22:25:00,2005-01-15 23:02:00,2005-01-15 23:31:00
2,1997-11-27 13:30:00,1997-11-27 14:00:00,14000,7000,N17E63,8113,X2.6,11/27,13:56,91,441,98,50,9632,2001-09-24 09:32:00,2001-09-24 10:38:00,2001-09-24 11:09:00
3,2001-09-24 10:45:00,2001-09-25 20:00:00,7000,30,S16E23,9632,X2.6,09/24,10:30,360,2402,Halo,49,0720,2005-01-15 22:25:00,2005-01-15 23:02:00,2005-01-15 23:31:00
4,2001-09-24 10:45:00,2001-09-25 20:00:00,7000,30,S16E23,9632,X2.6,09/24,10:30,360,2402,Halo,50,9632,2001-09-24 09:32:00,2001-09-24 10:38:00,2001-09-24 11:09:00
5,2005-01-15 23:00:00,2005-01-15 00:00:00,3000,40,N15W05,10720,X2.6,01/15,23:06,360,2861,Halo,49,0720,2005-01-15 22:25:00,2005-01-15 23:02:00,2005-01-15 23:31:00
6,2005-01-15 23:00:00,2005-01-15 00:00:00,3000,40,N15W05,10720,X2.6,01/15,23:06,360,2861,Halo,50,9632,2001-09-24 09:32:00,2001-09-24 10:38:00,2001-09-24 11:09:00
7,1998-05-06 08:25:00,1998-05-06 08:35:00,14000,5000,S11W65,8210,X2.7,05/06,08:29,190,1099,309,46,2339,2015-05-05 22:05:00,2015-05-05 22:11:00,2015-05-05 22:15:00
8,1998-05-06 08:25:00,1998-05-06 08:35:00,14000,5000,S11W65,8210,X2.7,05/06,08:29,190,1099,309,47,0488,2003-11-03 01:09:00,2003-11-03 01:30:00,2003-11-03 01:45:00
9,1998-05-06 08:25:00,1998-05-06 08:35:00,14000,5000,S11W65,8210,X2.7,05/06,08:29,190,1099,309,48,8210,1998-05-06 07:58:00,1998-05-06 08:09:00,1998-05-06 08:20:00


In [54]:
pd.merge(frame1, frame2, on = 'importance',how='outer')

,Start_Datetime,End_Datetime,startFrequency,endFrequency,flare_Location,flare_region,importance,CME_Date,CME_Time,width,speed,CPA,Rank,Region,Start_time,Max_time,End_time
0,1997-04-01 14:00:00,1997-04-01 14:15:00,8000,4000,S25E16,8026,M1.3,04/01,15:18,79,312,74,NaN,NaN,NaT,NaT,NaT
1,1997-05-21 20:20:00,1997-05-21 22:00:00,5000,500,N05W12,8040,M1.3,05/21,21:00,165,296,263,NaN,NaN,NaT,NaT,NaT
2,2000-02-08 09:05:00,2000-02-11 02:20:00,12000,20,N25E26,8858,M1.3,02/08,09:30,360,1079,Halo,NaN,NaN,NaT,NaT,NaT
3,2000-02-17 20:42:00,2000-02-18 22:12:00,14000,100,S29E07,8872,M1.3,02/17,21:30,360,728,Halo,NaN,NaN,NaT,NaT,NaT
4,2011-10-21 13:15:00,2011-10-21 13:50:00,16000,6500,N05W79,11319,M1.3,10/21,13:25,109,317,252,NaN,NaN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,NaT,NaT,NaN,NaN,NaN,NaN,X8.2,NaN,NaN,NaN,NaN,NaN,11,2673,2017-09-10 15:35:00,2017-09-10 16:06:00,2017-09-10 16:31:00
382,NaT,NaT,NaN,NaN,NaN,NaN,X6.5,NaN,NaN,NaN,NaN,NaN,14,0930,2006-12-06 18:29:00,2006-12-06 18:47:00,2006-12-06 19:00:00
383,NaT,NaT,NaN,NaN,NaN,NaN,X4,NaN,NaN,NaN,NaN,NaN,26,9236,2000-11-26 16:34:00,2000-11-26 16:48:00,2000-11-26 16:56:00
384,NaT,NaT,NaN,NaN,NaN,NaN,X3.7,NaN,NaN,NaN,NaN,NaN,30,8384,1998-11-22 06:30:00,1998-11-22 06:42:00,1998-11-22 06:49:00


## Exercise

In [55]:
## You are required to reterive a table in the page that provided below
## First step request the page and pass the text of your page to a bs4 object
link='http://sideeffects.embl.de/drugs/16156130/'

##################YourSol##################
pg = requests.get(link,headers=headers)
pg_text = pg.text
exe_page = BeautifulSoup(pg_text,'html.parser')

In [56]:
## Print the first 2000 elements in the page using bs4.prettify

##################YourSol##################
exe_page.prettify()[:2000]

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="width=device-width, initial-scale=1" name="viewport"/>\n  <meta content="Ivica Letunic" name="author"/>\n  <meta content="drug side effect resource" name="description"/>\n  <title>\n   Side effect information for Exenatide\n  </title>\n  <link href="/media/bs/css/bootstrap.min.css" rel="stylesheet"/>\n  <link href="//netdna.bootstrapcdn.com/font-awesome/4.0.3/css/font-awesome.css" rel="stylesheet"/>\n  <script src="/media/olib.js" type="text/javascript">\n  </script>\n  <script src="/media/jquery-1.11.2.min.js" type="text/javascript">\n  </script>\n  <script src="/media/jquery.cookie.js" type="text/javascript">\n  </script>\n  <script src="/media/panel/jquery-ui.min.js" type="text/javascript">\n  </script>\n  <link href="/media/panel/vendor/jquery-ui-1.11.4.custom/jquery-ui.min.css" rel="stylesheet">\n   <script src="/media/panel/jspanel/jquery.jspanel.m

In [57]:
## Find the tables in the page and print the number of tables

##################YourSol##################
len(exe_page.find_all("table"))

8

In [58]:
## Find first table and print the headers of table

##################YourSol##################
table=exe_page.find("table")
x=table.find_all('th')
for i in x:
    print(i.text)


Side effect


Data for drug


Placebo


Labels




In [59]:
## Create a dataframe that the columns names are the headers, and the rows are the rows of scrapped table

##################YourSol##################

Sideeffect=[]
Datafordrug=[]
Placebo=[]
Labels=[]
rows=table.find_all('tr')
i = 0 
for row in rows:
    if i == 0: # skip the header
        i += 1
        continue
    cols = row.find_all('td')
    Sideeffect.append(cols[0].text)
    Datafordrug.append(cols[1].text)
    Placebo.append(cols[2].text)
    Labels.append(cols[3].text)

In [60]:
pd.DataFrame({'Sideeffect':Sideeffect,'Datafordrug':Datafordrug,'Placebo':Placebo,'Labels':Labels})

,Sideeffect,Datafordrug,Placebo,Labels
0,\n,\n1\n,\n2\n,\n3\n
1,\nNausea \n,\npostmarketing — 1.3% - 44%\n,\n0% - 18%\n,\nx\n
2,\nDiarrhoea \n,\npostmarketing — 3.59% - 20%\n,\n3% - 8%\n,\nx\n
3,\nUpper respiratory tract infection \n,\n1.35% - 17.2%\n,\n,\n
4,\nFeeling jittery\n,\n9%\n,\n4%\n,\nx\n
...,...,...,...,...
100,\nNeuroglycopenia\n,\n,\n,\n
101,\nInjection site dermatitis\n,\n,\n,\n
102,\nBlood glucose decreased \n,\n,\n,\n
103,\nInfestation NOS\n,\n,\n,\n
